In [220]:
import numpy as np
import pandas as pd
import re

In [221]:
brickset = pd.read_csv('https://raw.githubusercontent.com/nk0305/COGS108_Repo/main/brickset.csv')
brickset = brickset.drop(columns = ['packaging', 'subtheme', 'rrp', 'ppp', 'notes'])
brickset.columns = ['Set_number', 'Name', 'Theme','Year','Pieces','No_of_minifigs','Minifig_list','Launch_and_Exit','newValue','usedValue','No_owned','No_want','Rating','Num_ratings','Num_reviews','Tags','Designer']
#Define a function to clean 'Launch_and_Exit' column

def clean_LnE(string):

    new_string = re.sub(r'\([^)]*\)', '', string)

    return new_string

#Apply the function to designated column

brickset['Launch_and_Exit'] = brickset['Launch_and_Exit'].astype('str')
brickset['Launch_and_Exit'] = brickset['Launch_and_Exit'].apply(clean_LnE)

#Separate launch and exit date

for date in brickset['Launch_and_Exit']:
 sep_string = re.split(" - ", date)

#create two new columns based on separated dates. Then, delete original "Launch_and_Exit" column

brickset['Launch_Date'] = sep_string[0]

brickset['Exit_Date'] = sep_string[1]

brickset = brickset.drop(columns = 'Launch_and_Exit')

#Finally, review the updated dataset

brickset.head()

,Set_number,Name,Theme,Year,Pieces,No_of_minifigs,Minifig_list,newValue,usedValue,No_owned,No_want,Rating,Num_ratings,Num_reviews,Tags,Designer,Launch_Date,Exit_Date
0,8630-1,Gold Hunt,Agents,2008,352.0,3.0,"['agt007', 'agt008', 'agt009']",~$87.35,~$22.75,3182.0,1188,4.0,95.0,11.0,"['Agent Fuse', 'Gold Tooth', 'Henchman', '4X4'...",['Mark Stafford'],1 Jul 2002,31 Dec 2003
1,8631-1,Jetpack Pursuit,Agents,2008,88.0,2.0,"['agt004', 'agt005']",~$36.18,~$10.11,4534.0,960,3.9,124.0,27.0,"['Agent Chase', 'Saw Fist', 'Aircraft', 'Dr In...",NaN,1 Jul 2002,31 Dec 2003
2,8632-1,Swamp Raid,Agents,2008,231.0,2.0,"['agt003', 'agt006']",~$51.25,~$17.15,3564.0,1085,3.9,106.0,18.0,"['Agent Charge', 'Break Jaw', 'Boat', 'Crimina...",['Mark Stafford'],1 Jul 2002,31 Dec 2003
3,8633-1,Speedboat Rescue,Agents,2008,340.0,3.0,"['agt001a', 'agt002', 'agt003']",~$90.56,~$35.70,2490.0,1239,3.9,75.0,9.0,"['Agent Chase', 'Agent Trace', 'Break Jaw', 'B...",['Raphael Pretesacque'],1 Jul 2002,31 Dec 2003
4,8634-1,Turbocar Chase,Agents,2008,498.0,3.0,"['agt001', 'agt008', 'agt015']",~$220.00,~$52.70,2439.0,1433,4.3,76.0,9.0,"['Agent Chase', 'Henchman', 'Spy Clops', 'Airc...",['Raphael Pretesacque'],1 Jul 2002,31 Dec 2003


In [230]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
# Cleaning
brickset = brickset.dropna()
brickset['Year'] = pd.to_numeric(brickset[brickset['Year'] != 'Buzz Lightyear']['Year'])
brickset = brickset[brickset['Year'] < 2024]

input_features = ['Theme', 'Pieces', 'No_of_minifigs', 'Rating']
output_features = ['No_want']
brickset_input = brickset[input_features]
brickset_output = brickset[output_features]

brickset_input['No_of_minifigs'] = brickset_input['No_of_minifigs'].apply(lambda x: 0 if np.isnan(x) else x)
encoder = OneHotEncoder(sparse_output=False)
brickset_encoder = encoder.fit_transform(brickset_input[['Theme']])
brickset_input = pd.concat([brickset_input,pd.DataFrame(brickset_encoder, columns=encoder.get_feature_names_out(), index=brickset_input.index)], axis=1)
brickset_input = brickset_input.drop('Theme', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(brickset_input, brickset_output, test_size=0.2, random_state=200)

C:\Users\Suhaib Chowdhury\AppData\Local\Temp\ipykernel_94272\177501234.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brickset_input['No_of_minifigs'] = brickset_input['No_of_minifigs'].apply(lambda x: 0 if np.isnan(x) else x)


In [231]:
# Create Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, Y_train)

ridge_reg_model = Ridge(alpha=0)
ridge_reg_model.fit(X_train, Y_train)

dt_reg_model = DecisionTreeRegressor()
dt_reg_model.fit(X_train,Y_train)

rf_reg_model = RandomForestRegressor()
rf_reg_model.fit(X_train,Y_train)

gb_reg_model = GradientBoostingRegressor()
gb_reg_model.fit(X_train,Y_train)

models = [linear_reg_model, ridge_reg_model, dt_reg_model, rf_reg_model, gb_reg_model]

C:\Users\Suhaib Chowdhury\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\Suhaib Chowdhury\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [232]:
# Testing
from sklearn.metrics import mean_squared_error

for model in models:
    train_pred = model.predict(X_train)
    print("Training Error: " + str(mean_squared_error(Y_train, train_pred, squared=False)))
    test_pred = model.predict(X_test)
    print("Testing Error: " + str(mean_squared_error(Y_test, test_pred, squared=False)))

Training Error: 0.06232797512142278
Testing Error: 0.07021751634978883
Training Error: 0.06233086851963599
Testing Error: 0.07020522288158457
Training Error: 0.0005466354114828788
Testing Error: 0.05957999236603102
Training Error: 0.02083208986297474
Testing Error: 0.0517848839345923
Training Error: 0.04325298106211922
Testing Error: 0.0529271058025812
